In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import statistics as stats # https://docs.python.org/3/library/statistics.html#statistics.fmean
#import scipy.stats as spstats
import matplotlib.pyplot as plt

### Validation & Normalization methods ###
from sklearn.model_selection import cross_validate, StratifiedKFold, RepeatedStratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

### ML models ###
from sklearn.linear_model import LogisticRegression, SGDClassifier # C1 loss: log_loss => LogisticRegression with SGD
from sklearn.linear_model import Perceptron # C2
from sklearn.svm import SVC # C3
from sklearn.svm import LinearSVC # C4
from sklearn.tree import DecisionTreeClassifier # C5
from sklearn.ensemble import RandomForestClassifier # C6
from sklearn.neural_network import MLPClassifier # C7

### Metrics ###
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, make_scorer
from imblearn.metrics import geometric_mean_score # https://imbalanced-learn.org/stable/references/generated/imblearn.metrics.geometric_mean_score.html
import time
import timeit # https://stackoverflow.com/questions/17579357/time-time-vs-timeit-timeit

### Pipeline ###
from sklearn.pipeline import make_pipeline , Pipeline # https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

### Analysis ###
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE, RFECV , mutual_info_classif

### Custom Modules ###
from functions.data_types import optimize_dtypes
from functions.dataframe_actions import df_info, df_clean, show_value_counts, fill_missing_values
from functions.ml_training import train_classifiers, train_classifiers_tuned, train_evaluate_single
from functions.dimensionality_reduction import apply_pca, plot_variance

### Other configurations ###
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'
# #import warnings library
# import warnings
# # ignore all warnings
# warnings.filterwarnings('ignore')

from sklearn import set_config
set_config(working_memory=1024*20) 

from sklearn.cluster import AgglomerativeClustering, FeatureAgglomeration
from sklearn.decomposition import PCA, SparsePCA, KernelPCA, TruncatedSVD

/home/lazaros/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
# to read .csv files from another directory
data_location = "../Data/" # "/<path>"

df_train = pd.read_csv(data_location + "train_filled_mapped.csv")
df_train = optimize_dtypes(df_train)
# df_train.head()

df_test = pd.read_csv(data_location + "test_filled_mapped.csv")
df_test = optimize_dtypes(df_test)


# Separate target variable from feature variables
X_train = df_train.drop('HadHeartAttack', axis=1, inplace=False)  # Features
y_train = df_train['HadHeartAttack']

# Separate target variable from feature variables
X_test = df_test.drop('HadHeartAttack', axis=1, inplace=False)  # Features
y_test = df_test['HadHeartAttack']   

In [3]:
original_dtypes = X_train.dtypes

X_train_scaled = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0)-X_train.min(axis=0))              # min max scale
# X_train_scaled = (X_train - X_train.mean())/X_train.std() # If we use StandardScaler, the feature names will be lost, so we do it mannually.

# x_scaled.hist(figsize=(16, 20), bins=30, edgecolor="black") # plot to show features after scaling
# plt.subplots_adjust()

# Cast back to original data types
for column, dtype in original_dtypes.items():
    X_train_scaled[column] = X_train_scaled[column].astype(dtype)


original_dtypes = X_test.dtypes

# Apply Min-Max scaling based on training set statistics
X_test_scaled = (X_test - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))

# Cast back to original data types
for column, dtype in original_dtypes.items():
    X_test_scaled[column] = X_test_scaled[column].astype(dtype)

In [4]:
pca = PCA(n_components=15, svd_solver='full', random_state=13).fit(X_train_scaled)
X_pca_train = pca.transform(X_train_scaled)
X_pca_test = pca.transform(X_test_scaled)

In [5]:
rf = RandomForestClassifier(criterion='entropy', n_jobs=-1, class_weight='balanced', random_state=13)
metrics_rf = train_evaluate_single(rf, X_pca_train, y_train, X_pca_test, y_test, classifier_name="RandomForest")

# Store metrics in a DataFrame
metrics_df_rf = pd.DataFrame([metrics_rf])
metrics_df_rf


,classifier,accuracy,balanced_accuracy,precision,recall,f1_score,roc_auc,fit_time
0,RandomForest,0.942713,0.538385,0.475259,0.082238,0.140214,0.8366,58.77455


In [7]:
svc = LinearSVC(kernel = 'rbf', gamma='scale', cache_size=10000, class_weight='balanced', random_state=13)
metrics_svc=train_evaluate_single(svc, X_pca_train, y_train, X_pca_test, y_test, classifier_name="LinearSVC")

# Store metrics in a DataFrame
metrics_df_svc = pd.DataFrame([metrics_svc])
metrics_df_svc
